In [54]:
'''
Stellar Evolution Predictor
(Using Mass, Metallicity, and Monte Carlo Uncertainty)

This notebook predict's a star's final evolutionary remnant using astrophysical approximations:

Inputs (from user):
- initial mass in solar masses
- metallicity (Z)

Outputs:
- Main-sequence lifetime
- Effective final mass (M_eff) after mass loss
- Remnant classification
- Monte Carlo probability distribution of final outcomes
- Mass loss distribution (Bar chart)
- Nucleosynthetic yield visualization (Bar chart)

'''

"\nStellar Evolution Predictor\n(Using Mass, Metallicity, and Monte Carlo Uncertainty)\n\nThis notebook predict's a star's final evolutionary remnant using astrophysical approximations:\n\nInputs (from user):\n- initial mass in solar masses\n- metallicity (Z)\n\nOutputs:\n- Main-sequence lifetime\n- Effective final mass (M_eff) after mass loss\n- Remnant classification\n- Monte Carlo probability distribution of final outcomes\n- Mass loss distribution (Bar chart)\n- Nucleosynthetic yield visualization (Bar chart)\n\n"

In [83]:
import numpy as np, matplotlib.pyplot as plt
from collections import Counter
import re
import pandas as pd
from io import StringIO
import glob
import os

In [ ]:
# PARSE ISOTOPE YIELD FILES
def parse_yield_file(path):
    ''' parse files with structure:
    H Table: (M=1.0,Z=0.01)
    ...
    Isotopes Yields ... ...
    H-1 ...
    H-2 ...
    Returns: mass, df with isotopes + yields

    fix: finds mass from file name instead
    read online lines that start with Element-Number
    takes the LAST numerical entry to avoid duplicates
    skips malformed/wrapped lines
    '''

    # read file
    with open(path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    # extract mass from filename
    mass = None
    for line in lines:
        if "M=" in line:
            text = line.strip()
            left = text.find("M=") + 2
            right = left
            while right < len(text) and (text[right].isdigit() or text[right] in ".+-eE"):
                right += 1
            mass = float(text[left:right])
            break

    
    isotopes, yields = [], []

    with open(path, errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            
            
            line_parts = line.split()
            isotp_name = line_parts[0]

            if "-" not in isotp_name:
                continue

            y = float(line_parts[1]) #yield will always be second entry no matter the format

            
            isotopes.append(isotp_name)
            yields.append(y)

    df = pd.DataFrame({"Isotopes": isotopes, "Yields": yields})
    df = df.drop_duplicates(subset="Isotopes", keep="first")

    return mass, df


In [126]:
# LOOP TO PARSE ALL FILES

def make_yield_grid(folder):
    dfs, masses = [], []

    for fname in os.listdir(folder):
        if not fname.endswith(".txt"):
            continue
        path = os.path.join(folder, fname)

        mass, df = parse_yield_file(path)
        colname = f"M{mass}"

        df = df.rename(columns={"Yields" : colname})

        dfs.append(df)
        masses.append(mass)

    combined_df = dfs[0]
    for df in dfs[1:]:
        combined_df = combined_df.merge(df, on="Isotopes", how="outer")

    combined_df = combined_df.set_index("Isotopes")

    col_order = sorted(combined_df.columns, key=lambda x: float(x[1:]))
    combined_df = combined_df[col_order]

    return combined_df



In [ ]:
df_Z00001 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z00001")
df_Z00002 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z00002")
df_Z0001 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z0001")
df_Z0006 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z0006")
df_Z001 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z001")
df_Z002 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z002")
df_Z003 = make_yield_grid("/Users/sarayu/python_decal_fa25/final_project/yield_data/Z003")

# df_Z0001, df_Z00002, df_Z0001, df_Z0006, df_Z001, df_Z002, df_Z003

# df_Z003

,M2.0,M3.0
Isotopes,,
Ag-101,6.673255e-100,1.490694e-99
Ag-102,6.673255e-100,1.490694e-99
Ag-103,6.673255e-100,1.490694e-99
Ag-104,6.673255e-100,1.490694e-99
Ag-105,6.673255e-100,1.490694e-99
Ag-106,6.673255e-100,1.490694e-99
Ag-107,7.721445e-10,1.695498e-09
Ag-108,6.673255e-100,1.490694e-99
Ag-109,8.387616e-10,3.097485e-09


In [ ]:
df

In [56]:
#INPUT

In [57]:
#MAIN SEQUENCE LIFETIME

In [58]:
#MASS LOSS FUNCTION FROM METALLICITY Z